In [ ]:
import os

In [ ]:
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/NLP/BERT')

#import data from kaggle on Google Colab by using Kaggle API

In [ ]:
!pip install -q kaggle 

In [ ]:
!pip install -q kaggle-cli 

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 5.3MB 9.5MB/s 
     |████████████████████████████████| 112kB 49.9MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 133kB 49.1MB/s 


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


#downloading data from kaggle competition to Google Drive

In [ ]:

!kaggle competitions download -c twitter-sentiment-analysis2

 82% 9.00M/11.0M [00:00<00:00, 24.9MB/s]
100% 11.0M/11.0M [00:00<00:00, 24.7MB/s]
  0% 0.00/3.67M [00:00<?, ?B/s]
100% 3.67M/3.67M [00:00<00:00, 122MB/s]


In [ ]:
!unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


#Importing Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df=pd.read_csv('train.csv',encoding='latin-1',header=None)

In [ ]:
df.head()

,0,1,2
0,ItemID,Sentiment,SentimentText
1,1,0,is so sad for my APL frie...
2,2,0,I missed the New Moon trail...
3,3,1,omg its already 7:30 :O
4,4,0,.. Omgaga. Im sooo im gunna CRy. I'...


#Adding some data

In [ ]:
df[2][0]='I am not feeling good here.'

In [ ]:
df[1][0]='0'
df[0][0]=0

In [ ]:
df.head()

,0,1,2
0,0,0,I am not feeling good here.
1,1,0,is so sad for my APL frie...
2,2,0,I missed the New Moon trail...
3,3,1,omg its already 7:30 :O
4,4,0,.. Omgaga. Im sooo im gunna CRy. I'...


In [ ]:
df1=df.copy()

In [ ]:
df.columns=['Id','Sentiment','Tweet_text']

In [ ]:
df.head()

,Id,Sentiment,Tweet_text
0,0,0,I am not feeling good here.
1,1,0,is so sad for my APL frie...
2,2,0,I missed the New Moon trail...
3,3,1,omg its already 7:30 :O
4,4,0,.. Omgaga. Im sooo im gunna CRy. I'...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99990 entries, 0 to 99989
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Id          99990 non-null  object
 1   Sentiment   99990 non-null  object
 2   Tweet_text  99990 non-null  object
dtypes: object(3)
memory usage: 2.3+ MB


In [ ]:
df.head()

,Id,Sentiment,Tweet_text
0,0,0,I am not feeling good here.
1,1,0,is so sad for my APL frie...
2,2,0,I missed the New Moon trail...
3,3,1,omg its already 7:30 :O
4,4,0,.. Omgaga. Im sooo im gunna CRy. I'...


In [ ]:
df.drop('Id',inplace=True,axis=1)

In [ ]:
df.head()

,Sentiment,Tweet_text
0,0,I am not feeling good here.
1,0,is so sad for my APL frie...
2,0,I missed the New Moon trail...
3,1,omg its already 7:30 :O
4,0,.. Omgaga. Im sooo im gunna CRy. I'...


In [ ]:
df.columns

Index(['Sentiment', 'Tweet_text'], dtype='object')

In [ ]:
df['Sentiment']=df['Sentiment'].apply(int)

In [ ]:
df['Sentiment'][0]

0

In [ ]:
X=df['Tweet_text'].values
y=df['Sentiment'].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val =train_test_split(X, y, test_size=0.1, random_state=101)

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [ ]:
!pip install transformers

In [ ]:
def text_preprocessing(text):
    
    
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    
    text = re.sub(r'&amp;', '&', text)

    
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


def preprocessing_for_bert(data):

    input_ids = []
    attention_masks = []

    for sent in data:

        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  
            add_special_tokens=True,        
            max_length=MAX_LEN,                 
            pad_to_max_length=True,            
            return_attention_mask=True ,
            truncation=True     
            )
        
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:

all_tweets = df.Tweet_text.values


encoded_tweets = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_tweets]

max_len = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len)

Max length:  316


In [ ]:

MAX_LEN = 64
import re


token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)


print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

Original:  I am not feeling good here.
Token IDs:  [101, 1045, 2572, 2025, 3110, 2204, 2182, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tokenizing data...


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)


batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel


class BertClassifier(nn.Module):

    def __init__(self, freeze_bert=False):

        super(BertClassifier, self).__init__()
       
        D_in, H, D_out = 768, 50, 2

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
           
            nn.Linear(H, D_out)
        )

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
       
       
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
       
        last_hidden_state_cls = outputs[0][:, 0, :]

       
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 46 µs, sys: 1e+03 ns, total: 47 µs
Wall time: 49.8 µs


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
 
    bert_classifier = BertClassifier(freeze_bert=False)

    
    bert_classifier.to(device)


    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    
                      eps=1e-8   
                      )
   
    total_steps = len(train_dataloader) * epochs

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, 
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
import random
import time


loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    
    print("Start training...\n")
    for epoch_i in range(epochs):
       
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        
        t0_epoch, t0_batch = time.time(), time.time()

        
        total_loss, batch_loss, batch_counts = 0, 0, 0

        
        model.train()

        
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
            
            model.zero_grad()

           
            logits = model(b_input_ids, b_attn_mask)

            
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            
            loss.backward()

            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            
            optimizer.step()
            scheduler.step()

            
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
               
                time_elapsed = time.time() - t0_batch

                
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        
        if evaluation == True:
           
            val_loss, val_accuracy = evaluate(model, val_dataloader)

           
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    
    model.eval()

    
    val_accuracy = []
    val_loss = []

    
    for batch in val_dataloader:
        
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        preds = torch.argmax(logits, dim=1).flatten()

        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
set_seed(42)    
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.661556   |     -      |     -     |   7.13   
   1    |   40    |   0.536703   |     -      |     -     |   6.65   
   1    |   60    |   0.530981   |     -      |     -     |   6.65   
   1    |   80    |   0.488778   |     -      |     -     |   6.68   
   1    |   100   |   0.467077   |     -      |     -     |   6.73   
   1    |   120   |   0.453356   |     -      |     -     |   6.82   
   1    |   140   |   0.453102   |     -      |     -     |   6.87   
   1    |   160   |   0.513905   |     -      |     -     |   6.90   
   1    |   180   |   0.390171   |     -      |     -     |   6.98   
   1    |   200   |   0.478004   |     -      |     -     |   7.02   
   1    |   220   |   0.481195   |     -      |     -     |   7.09   
   1    |   240   |   0.494547   |     -      |     -     |   7.17   